# Chapter 8: The Model Context Protocol (MCP)

## A Universal Language for AI Agents

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPOSITORY/blob/main/notebooks/08-Model-Context-Protocol.ipynb)

Welcome to a more advanced, forward-looking chapter. We've seen how to build agents with different frameworks, but how do we make them talk to each other? In this chapter, we'll explore the **Model Context Protocol (MCP)**, a proposed standard for enabling interoperability between diverse AI agents.

## 🗣️ The Problem: The Agentic Tower of Babel

Imagine you have a powerful research agent built with LangGraph and a creative writing agent built with CrewAI. You want them to collaborate. The problem is, they have completely different ways of managing their internal state, memory, and tools. They don't speak the same language.

This lack of a common communication standard makes it incredibly difficult to build truly modular and interoperable multi-agent systems.

## 💡 The Solution: A Shared 'Whiteboard'

The **Model Context Protocol (MCP)** proposes a simple but powerful solution: a central **Context Server**. Think of it as a shared digital whiteboard.

- **Agents** can write information to the whiteboard.
- **Agents** can read information from the whiteboard.

Instead of talking to each other directly, agents communicate *through* this shared context. The protocol defines a standard way to structure this information (a **schema**), ensuring that all agents can understand it. This enables agents built with any framework to collaborate seamlessly.

### Hands-On: Building a Mini MCP Server

Since MCP is still an emerging concept, there aren't many off-the-shelf libraries. So, we'll build our own simplified version from scratch using `FastAPI` to understand the core principle.

In [ ]:
# Step 1: Install and Setup
!pip install fastapi uvicorn requests nest_asyncio python-dotenv

import nest_asyncio
nest_asyncio.apply() # Allows Uvicorn to run in a notebook

In [ ]:
# Step 2: The MCP Server Code
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Dict, Any

app = FastAPI()

# This dictionary is our simple, in-memory context store
shared_context: Dict[str, Any] = {}

class ContextUpdate(BaseModel):
    key: str
    value: Any

@app.get("/context")
def get_context():
    """Reads the entire shared context."""
    return shared_context

@app.post("/context")
def update_context(update: ContextUpdate):
    """Updates a specific key in the shared context."""
    shared_context[update.key] = update.value
    return {"status": "success", "updated_key": update.key}


In [ ]:
# Step 3: Run the Server in the Background
import uvicorn
import threading
import time

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run the server in a separate thread so it doesn't block the notebook
server_thread = threading.Thread(target=run_server)
server_thread.start()
time.sleep(2) # Give the server a moment to start

print("MCP Server is running in the background on port 8000.")

### Creating MCP-Aware "Agents"

Now that our server is running, let's create two simple functions that act as our agents. They will communicate with the server using `requests`.

In [ ]:
# Step 4: Define the Agents
import requests
import json

SERVER_URL = "http://localhost:8000"

def researcher_agent():
    print("--- Researcher Agent Running ---")
    # In a real scenario, this would come from a web search or DB lookup
    research_data = "The Model Context Protocol (MCP) is a proposed standard for agent interoperability."
    
    print("  -> Found data, updating context server...")
    response = requests.post(
        f"{SERVER_URL}/context", 
        json={"key": "research_summary", "value": research_data}
    )
    print(f"  -> Server response: {response.json()}")

def writer_agent():
    print("\n--- Writer Agent Running ---")
    print("  -> Reading data from context server...")
    response = requests.get(f"{SERVER_URL}/context")
    context = response.json()
    summary = context.get("research_summary", "No summary found.")
    
    print("  -> Generating article based on context:")
    article = f"**Blog Post Title: The Future of AI is Collaborative**\n\n{summary}"
    print(article)

In [ ]:
# Step 5: Simulate the Workflow

# Initial state of the server
print(f"Initial server context: {requests.get(f'{SERVER_URL}/context').json()}")

# Run the first agent
researcher_agent()

# Check the server state now
print(f"\nServer context after researcher: {requests.get(f'{SERVER_URL}/context').json()}")

# Run the second agent
writer_agent()

## ✅ Why This is a Big Deal

Notice that the `researcher_agent` and `writer_agent` never called each other directly. They only interacted with the central context server. This **decoupling** is the key.

With a standardized protocol like MCP, you could have agents written in different languages, using different frameworks (CrewAI, LangGraph, AutoGen), all seamlessly collaborating on a single task. This is the foundation for a truly open and modular ecosystem of AI agents.

Congratulations on completing the most forward-thinking chapter in this course! In the final chapter, we'll undertake a **capstone project** to combine everything you've learned into one impressive, modular agentic system.